In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import LocallyLinearEmbedding

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from hyperopt import fmin, tpe, hp
from bayes_opt import BayesianOptimization

%matplotlib inline

# SVR optimization methods

In [2]:
def bayesian_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda epsilon: cross_val_score(
            LinearSVR(C=100, epsilon=epsilon, max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'epsilon': (0.0, 5.0)},
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']['epsilon']

def hyperopt_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'svr',
            'epsilon': hp.uniform('svr_epsilon', 0.0, 5.0)
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LinearSVR(C=100, epsilon=args['epsilon'], max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )

    return best['svr_epsilon']

def evaulate_best_svr_argument(X, y, eps_vals: list, cv=6):
    scores = []
    for e in eps_vals:
        scores.append(cross_val_score(
            LinearSVR(C=100, epsilon=e, max_iter=200),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean())
        
    return eps_vals[np.argmax(scores)]

# RF regressor optimization code

In [3]:
# hyperopt for lightgbm shows terrible results
def hyperopt_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'lightgbm',
            'feature_fraction': hp.uniform('feature_fraction', 0.05, 0.95),
            'bagging_fraction': hp.uniform('bagging_fraction', 0.05, 0.95),
            'bagging_freq': hp.uniform('bagging_freq', 1, 50),
            'n_estimators': hp.uniform('n_estimators', 5, 50),
            #'max_bin': hp.uniform('max_bin', )
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=args['feature_fraction'], 
                bagging_freq=int(args['bagging_freq']), 
                bagging_fraction=args['bagging_fraction'],
                n_estimators=int(args['n_estimators'])
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )
    
    return best

def bayesian_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda feature_fraction, bagging_freq, bagging_fraction, n_estimators: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=feature_fraction, 
                bagging_freq=int(bagging_freq), 
                bagging_fraction=bagging_fraction,
                n_estimators=int(n_estimators)
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'feature_fraction': (0.05, 0.95),
         'bagging_fraction': (0.05, 0.95),
         'bagging_freq': (1, 50),
         'n_estimators': (5, 50) },
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']#['C']

## Facebook Comment Volume Dataset

In [64]:
df = pd.read_csv('../datasets/facebook_comments.csv', index_col=0)

In [65]:
print('dataset size: {}'.format(df.shape))

dataset size: (40949, 54)


In [66]:
# encoding categorical variables
# H Local - category
# Post Promotion Status - category
# Base Time - time variable
# Page Category - category

H_Local = OneHotEncoder().fit_transform(df['H Local'].values.reshape(-1, 1)).todense()
Post_Promotion_Status = OneHotEncoder().fit_transform(df['Post Promotion Status'].values.reshape(-1, 1)).todense()
Base_Time = OneHotEncoder().fit_transform(df['Base Time'].values.reshape(-1, 1)).todense()
Page_Category = OneHotEncoder().fit_transform(df['Page Category'].values.reshape(-1, 1)).todense()

In [67]:
y = df.Target.values
X = df.drop(['H Local', 'Post Promotion Status', 'Base Time', 'Page Category', 'Target'], axis=1).values
X = np.hstack([X, H_Local, Post_Promotion_Status, Base_Time, Page_Category])

In [68]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (40949, 228)


### Training model with LLE-processed data

In [70]:
# number of components is fixed to 10
lle = LocallyLinearEmbedding(n_components=10)
X_lle = lle.fit_transform(X)

In [71]:
X_train, X_val, y_train, y_val = train_test_split(X_lle, y, test_size=0.1)

### * Estimating SVR penalty

In [72]:
eps_hyperopt = hyperopt_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.18355513]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.33743858]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'wa

In [73]:
eps_opt = evaulate_best_svr_argument(X_train, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [74]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.82401937]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-11.43000153]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [75]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [76]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [77]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_train, y_train):
        X_crossval_train, X_crossval_test = X_train[train_ix], X_train[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [78]:
print('for decorrelated data using LLE (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using LLE (averate scores)
linear, MSE: 1342.0992772718141
svr, MSE: 1351.3679268771216
forest, MSE: 1278.6361101356306


In [79]:
print('for decorrelated data using LLE (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using LLE (validation scores):
linear, MSE: 485.2666959994564
svr, MSE: 486.217324974968
forest, MSE: 469.0029669257356


## Parkinsons Telemonitoring Data Set

In [40]:
df = pd.read_csv('../datasets/parkinsons_updrs.data')

In [41]:
print('dataset size: {}'.format(df.shape))

dataset size: (5875, 22)


In [42]:
subject = df['subject#'].unique().tolist()
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, int(i['subject#']) - 1] = 1

In [43]:
# replacing categorical features with binary values
y = df.total_UPDRS.values
X = df.drop(['motor_UPDRS', 'total_UPDRS', 'subject#'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [44]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (5875, 61)


### Training model with LLE-processed data

In [45]:
# number of components is fixed to 10
lle = LocallyLinearEmbedding(n_components=10)
X_lle = lle.fit_transform(X)

In [46]:
X_train, X_val, y_train, y_val = train_test_split(X_lle, y, test_size=0.1)

### * Estimating SVR penalty

In [47]:
eps_hyperopt = hyperopt_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-211.71980453]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 86, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-25.71723676]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 62, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.57396379]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 85, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: Us

In [48]:
eps_opt = evaulate_best_svr_argument(X_train, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [49]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

In [50]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [51]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [52]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_train, y_train):
        X_crossval_train, X_crossval_test = X_train[train_ix], X_train[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [53]:
print('for decorrelated data using LLE (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using LLE (averate scores)
linear, MSE: 113.98378600234898
svr, MSE: 114.03933957421967
forest, MSE: 86.16009321480038


In [54]:
print('for decorrelated data using LLE (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using LLE (validation scores):
linear, MSE: 110.37065046382854
svr, MSE: 110.45969152087248
forest, MSE: 87.24361821589834


## Energy efficiency Data Set

In [10]:
df = pd.read_excel('../datasets/ENB2012_data.xlsx')

In [11]:
print('dataset size: {}'.format(df.shape))

dataset size: (768, 10)


In [12]:
subject = df['X6'].unique().tolist()
subject_map = dict(zip(subject, range(len(subject))))
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, subject_map[i['X6']]] = 1

In [13]:
# replacing categorical features with binary values
y = df.Y1.values
X = df.drop(['Y1', 'Y2', 'X6'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [14]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (768, 11)


### Training model with LLE-processed data

In [15]:
# number of components is fixed to 4
lle = LocallyLinearEmbedding(n_components=4)
X_lle = lle.fit_transform(X)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_lle, y, test_size=0.1)

### * Estimating SVR penalty

In [17]:
eps_hyperopt = hyperopt_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_train, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.09471816]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [18]:
eps_opt = evaulate_best_svr_argument(X_train, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [19]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

In [20]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [21]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [22]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_train, y_train):
        X_crossval_train, X_crossval_test = X_train[train_ix], X_train[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [23]:
print('for decorrelated data using LLE (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using LLE (averate scores)
linear, MSE: 65.8845210107275
svr, MSE: 68.02664499125626
forest, MSE: 39.824307443025546


In [24]:
print('for decorrelated data using LLE (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using LLE (validation scores):
linear, MSE: 78.7676925666274
svr, MSE: 77.51319167924918
forest, MSE: 34.395977913294054
